In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [2]:
# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import OpenAI
from trulens_eval import Tru

tru = Tru()
tru.reset_database()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
"You are an expert in managing YouTube addiction, with a deep understanding of addiction psychology, goal-setting strategies, and behavior modification techniques. Your task is to create a personalized 30-day improvement plan for individuals seeking to balance their digital consumption with personal growth. You will receive input regarding the individual's personal goals and passions, alongside their specific goals for reducing YouTube usage and content they aim to avoid. Based on this information, you are to provide a detailed plan that includes:

Initial Assessment: Summarize the individual's current situation based on their input, highlighting key areas for improvement in their YouTube consumption habits and how these relate to their personal goals and passions.
Goal-Setting: Outline specific, measurable, achievable, relevant, and time-bound (SMART) goals for both reducing YouTube addiction and advancing towards their personal objectives.
Daily Action Steps: Provide a day-by-day guide for the 30-day period, with each day's activities designed to gradually reduce YouTube consumption and encourage progress on their personal goals. Include practical tips for avoiding triggers, managing cravings, and finding healthier alternatives.
Educational Insights: Integrate brief educational snippets about the psychology of addiction and the role of dopamine in habit formation, tailored to help the individual understand and navigate their journey.
Motivational Support: Offer motivational messages and affirmations to encourage persistence, resilience, and self-compassion throughout the 30-day plan.
Remember to approach this task with empathy, offering guidance that is both realistic and supportive, considering the individual's unique circumstances and challenges."
""" 


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])



In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

def gpt35_turbo(user_input):
    gpt35_turbo = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
    chain = prompt | gpt35_turbo 
    return chain.invoke({"input": question})

def gpt4_turbo(question):
    gpt4_turbo = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    chain = prompt | gpt4_turbo 
    return chain.invoke({"input": question})

In [5]:
# OpenAI based feedback function collection class
openai_provider = OpenAI()

# Relevance feedback function using openai
f_relevance = Feedback(openai_provider.relevance, feedback_mode = "deferred").on_input_output()

# Conciseness feedback function using openai
f_conciseness = Feedback(openai_provider.conciseness, feedback_mode = "deferred").on_output()

# Stereotypes feedback function using openai
f_stereotypes = Feedback(openai_provider.stereotypes, feedback_mode = "deferred").on_input_output()

feedbacks = [f_relevance, f_conciseness, f_stereotypes]

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In conciseness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In stereotypes, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In stereotypes, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [6]:
from trulens_eval import TruBasicApp
gpt35_turbo_recorder = TruBasicApp(gpt35_turbo, app_id="gpt-3.5-turbo", feedbacks=feedbacks)
gpt4_turbo_recorder = TruBasicApp(gpt4_turbo, app_id="gpt-4-turbo", feedbacks=feedbacks)

In [7]:
user_input_1 = """
Personal Goal and Passion: "I'm passionate about painting and aspire to become a professional artist. My goal is to complete at least one painting per week, but I find myself spending hours on YouTube watching tutorials that I seldom put into practice."
Goal to Improve Addiction: "I want to reduce my YouTube usage, specifically the time spent on endless tutorial loops without actual practice. I aim to limit my consumption of art tutorials to only two hours a week, using the rest of my time for hands-on painting."
"""

user_input_2 = """
Personal Goal and Passion: "Fitness is my biggest passion, and I'm training for a half-marathon. However, I often find myself watching fitness vlogs and motivational videos on YouTube instead of going out for my runs or doing my workouts."
Goal to Improve Addiction: "I need to cut down on passive consumption of fitness content and spend more actual time training. My goal is to replace at least half of my YouTube viewing time with real-life training sessions, avoiding fitness vlogs and focusing only on necessary training videos."
"""

user_input_3 = """
Personal Goal and Passion: "I'm a student with upcoming exams in three major subjects. While I start on YouTube looking for study tips or educational content, I end up watching unrelated videos for hours, which is hampering my study schedule."
Goal to Improve Addiction: "My aim is to use YouTube solely for educational purposes related to my exams, avoiding any content not directly linked to my study topics. I want to limit my daily YouTube usage to a maximum of one hour, strictly for educational videos that contribute to my exam preparation."
"""

user_input_4 = """
Personal Goal and Passion: "Fitness is my ultimate passion, and I'm working towards running a marathon. Despite my dedication, I find myself endlessly browsing through fitness vlogs and motivational speeches on YouTube, which eats into my training time."
Goal to Improve Addiction: "My objective is to significantly cut down on my YouTube consumption, especially videos that are motivational but not directly related to my training regimen. I aim to restrict my viewing to only one hour per week strictly for instructional running and diet advice, reallocating the saved time to physical training and preparation for the marathon."
"""

user_input_5 = """
Personal Goal and Passion: "I am deeply interested in programming and aim to enhance my coding skills to build my own tech startup. My goal is to develop a new software project every month, but I often get sidetracked watching tech talks and tutorials on YouTube without actual coding."
Goal to Improve Addiction: "I wish to curb my YouTube usage, particularly the excessive time spent watching tech-related content that doesn't directly contribute to my coding practice. I plan to limit my viewing of such content to just three hours per week, dedicating the bulk of my time to hands-on software development and coding exercises."
"""

prompts = [
    user_input_1,
    user_input_2,
    user_input_3,
    user_input_4,
    user_input_5
]

In [8]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.204:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [9]:
with gpt35_turbo_recorder as recording:
    for question in prompts:
        print(question)
        gpt35_turbo_recorder.app(question)


Personal Goal and Passion: "I'm passionate about painting and aspire to become a professional artist. My goal is to complete at least one painting per week, but I find myself spending hours on YouTube watching tutorials that I seldom put into practice."
Goal to Improve Addiction: "I want to reduce my YouTube usage, specifically the time spent on endless tutorial loops without actual practice. I aim to limit my consumption of art tutorials to only two hours a week, using the rest of my time for hands-on painting."



/Users/joeywang/Documents/side_projects/youtube-addiction/lablab/coaching-program/new-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "Fitness is my biggest passion, and I'm training for a half-marathon. However, I often find myself watching fitness vlogs and motivational videos on YouTube instead of going out for my runs or doing my workouts."
Goal to Improve Addiction: "I need to cut down on passive consumption of fitness content and spend more actual time training. My goal is to replace at least half of my YouTube viewing time with real-life training sessions, avoiding fitness vlogs and focusing only on necessary training videos."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "I'm a student with upcoming exams in three major subjects. While I start on YouTube looking for study tips or educational content, I end up watching unrelated videos for hours, which is hampering my study schedule."
Goal to Improve Addiction: "My aim is to use YouTube solely for educational purposes related to my exams, avoiding any content not directly linked to my study topics. I want to limit my daily YouTube usage to a maximum of one hour, strictly for educational videos that contribute to my exam preparation."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "Fitness is my ultimate passion, and I'm working towards running a marathon. Despite my dedication, I find myself endlessly browsing through fitness vlogs and motivational speeches on YouTube, which eats into my training time."
Goal to Improve Addiction: "My objective is to significantly cut down on my YouTube consumption, especially videos that are motivational but not directly related to my training regimen. I aim to restrict my viewing to only one hour per week strictly for instructional running and diet advice, reallocating the saved time to physical training and preparation for the marathon."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "I am deeply interested in programming and aim to enhance my coding skills to build my own tech startup. My goal is to develop a new software project every month, but I often get sidetracked watching tech talks and tutorials on YouTube without actual coding."
Goal to Improve Addiction: "I wish to curb my YouTube usage, particularly the excessive time spent watching tech-related content that doesn't directly contribute to my coding practice. I plan to limit my viewing of such content to just three hours per week, dedicating the bulk of my time to hands-on software development and coding exercises."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.


In [10]:
with gpt4_turbo_recorder as recording:
    for question in prompts:
        print(question)
        gpt4_turbo_recorder.app(question)


Personal Goal and Passion: "I'm passionate about painting and aspire to become a professional artist. My goal is to complete at least one painting per week, but I find myself spending hours on YouTube watching tutorials that I seldom put into practice."
Goal to Improve Addiction: "I want to reduce my YouTube usage, specifically the time spent on endless tutorial loops without actual practice. I aim to limit my consumption of art tutorials to only two hours a week, using the rest of my time for hands-on painting."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "Fitness is my biggest passion, and I'm training for a half-marathon. However, I often find myself watching fitness vlogs and motivational videos on YouTube instead of going out for my runs or doing my workouts."
Goal to Improve Addiction: "I need to cut down on passive consumption of fitness content and spend more actual time training. My goal is to replace at least half of my YouTube viewing time with real-life training sessions, avoiding fitness vlogs and focusing only on necessary training videos."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "I'm a student with upcoming exams in three major subjects. While I start on YouTube looking for study tips or educational content, I end up watching unrelated videos for hours, which is hampering my study schedule."
Goal to Improve Addiction: "My aim is to use YouTube solely for educational purposes related to my exams, avoiding any content not directly linked to my study topics. I want to limit my daily YouTube usage to a maximum of one hour, strictly for educational videos that contribute to my exam preparation."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "Fitness is my ultimate passion, and I'm working towards running a marathon. Despite my dedication, I find myself endlessly browsing through fitness vlogs and motivational speeches on YouTube, which eats into my training time."
Goal to Improve Addiction: "My objective is to significantly cut down on my YouTube consumption, especially videos that are motivational but not directly related to my training regimen. I aim to restrict my viewing to only one hour per week strictly for instructional running and diet advice, reallocating the saved time to physical training and preparation for the marathon."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.



Personal Goal and Passion: "I am deeply interested in programming and aim to enhance my coding skills to build my own tech startup. My goal is to develop a new software project every month, but I often get sidetracked watching tech talks and tutorials on YouTube without actual coding."
Goal to Improve Addiction: "I wish to curb my YouTube usage, particularly the excessive time spent watching tech-related content that doesn't directly contribute to my coding practice. I plan to limit my viewing of such content to just three hours per week, dedicating the bulk of my time to hands-on software development and coding exercises."



Unsure what the main output string is for the call to _call with return type <class 'langchain_core.messages.ai.AIMessage'>.
